<a href="https://colab.research.google.com/github/sfansaria/CrimePredictionUsingLSTM-RandomForestClassifier-RandomForestRegressor-ARIMA/blob/main/Crime_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [254]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [255]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import math


import plotly.graph_objects as go



from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
#from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [256]:

crime_df = pd.read_csv('/content/drive/My Drive/ml_datasets/Crime_Data_from_2020_to_Present.csv')

print(crime_df.head(5))


       DR_NO               Date Rptd                DATE OCC  TIME OCC  AREA  \
0  190326475  03/01/2020 12:00:00 AM  03/01/2020 12:00:00 AM      2130     7   
1  200106753  02/09/2020 12:00:00 AM  02/08/2020 12:00:00 AM      1800     1   
2  200320258  11/11/2020 12:00:00 AM  11/04/2020 12:00:00 AM      1700     3   
3  200907217  05/10/2023 12:00:00 AM  03/10/2020 12:00:00 AM      2037     9   
4  220614831  08/18/2022 12:00:00 AM  08/17/2020 12:00:00 AM      1200     6   

   AREA NAME  Rpt Dist No  Part 1-2  Crm Cd  \
0   Wilshire          784         1     510   
1    Central          182         1     330   
2  Southwest          356         1     480   
3   Van Nuys          964         1     343   
4  Hollywood          666         2     354   

                                Crm Cd Desc  ... Status   Status Desc  \
0                          VEHICLE - STOLEN  ...     AA  Adult Arrest   
1                     BURGLARY FROM VEHICLE  ...     IC   Invest Cont   
2                

In [257]:
dict_zone = {}
for index, row in crime_df.iterrows():
    if( not (int(row['AREA']) in dict_zone)):
        dict_zone[row['AREA']] = row['AREA NAME']
dict_zone

{7: 'Wilshire',
 1: 'Central',
 3: 'Southwest',
 9: 'Van Nuys',
 6: 'Hollywood',
 18: 'Southeast',
 13: 'Newton',
 19: 'Mission',
 2: 'Rampart',
 4: 'Hollenbeck',
 10: 'West Valley',
 8: 'West LA',
 20: 'Olympic',
 21: 'Topanga',
 11: 'Northeast',
 12: '77th Street',
 14: 'Pacific',
 15: 'N Hollywood',
 5: 'Harbor',
 16: 'Foothill',
 17: 'Devonshire'}

#Zona

In [258]:
zonaUnderExam_id = 14
zonaUnderExam_name = dict_zone[zonaUnderExam_id]
print("ID =",zonaUnderExam_id, "Name=",zonaUnderExam_name)

ID = 14 Name= Pacific


In [259]:
crime_df['month'] = pd.to_datetime(crime_df['DATE OCC']).dt.strftime('%m-%Y')
crime_df.head(5)

<ipython-input-259-9029b69add4d>:1: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,month
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506,03-2020
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628,02-2020
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002,11-2020
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387,03-2020
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277,08-2020


In [260]:
Zone_Month = crime_df.groupby(["AREA","month"]).size().reset_index(name='Count')
Zone_Month.head(5)

,AREA,month,Count
0,1,01-2020,1211
1,1,01-2021,886
2,1,01-2022,1290
3,1,01-2023,1550
4,1,01-2024,1428


In [261]:
Zone_Month.tail(4)

,AREA,month,Count
1046,21,12-2020,649
1047,21,12-2021,815
1048,21,12-2022,803
1049,21,12-2023,803


In [262]:
Zone_Month.shape

(1050, 3)

In [263]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i+look_back, 0])
    return np.array(dataX), np.array(dataY)

In [264]:
AREA = zonaUnderExam_id

dataset = Zone_Month[(Zone_Month['AREA']==AREA)]
print(dataset.head(1))

drop_attributes = ['AREA','month']
dataset = dataset.drop(columns=drop_attributes)

print(dataset.head(1))
print("length of the dataset:",len(dataset))

dataset = dataset.values
dataset = dataset.astype('int32')
dataset

     AREA    month  Count
650    14  01-2020    944
     Count
650    944
length of the dataset: 50


array([[ 944],
       [ 982],
       [1118],
       [1122],
       [ 959],
       [ 919],
       [ 997],
       [ 959],
       [1050],
       [ 434],
       [ 884],
       [1046],
       [1213],
       [1128],
       [ 877],
       [1039],
       [1162],
       [1168],
       [ 906],
       [1166],
       [1138],
       [1106],
       [ 949],
       [1070],
       [1017],
       [1159],
       [1065],
       [1152],
       [1097],
       [1219],
       [1073],
       [1186],
       [1106],
       [1165],
       [1019],
       [1061],
       [ 988],
       [1192],
       [ 996],
       [1070],
       [1159],
       [1162],
       [ 975],
       [1056],
       [ 990],
       [1088],
       [ 959],
       [1094],
       [1080],
       [1073]], dtype=int32)

In [265]:
def moving_average(a, n=3):
  ret = np.cumsum(a, dtype=float)
  ret[n:] = ret[n:] - ret[:-n]
  return ret[n-1:]/n

In [266]:
list_graphs = []

graph_1 = go.Scatter(x=list(range(0,len(dataset))),y=dataset.flatten(),name="Dataset")
list_graphs.append(graph_1)

average = moving_average(dataset)

graph_2 = go.Scatter(x=list(range(0,len(dataset))),y=average,name="Average")
list_graphs.append(graph_2)

fig = go.Figure(data=list_graphs)
fig.update_layout(width=1024,height=500,title='Dataset of crimes in ' + dict_zone[AREA], xaxis=dict(title='Month'),yaxis=dict(title='Number of Crimes'))
fig.show()


In [267]:
len(dataset)

50

#MinMax Scaler to rescale the dataset and it gives more attention on outliers

In [268]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)


#Scale the dataset

#scaler = RobustScaler()
#dataset = scaler.fit_transform(dataset)

#Split the dataset with 70% for training and 30% for testing.

In [269]:
train_size = int(len(dataset)*0.7)
test_size = len(dataset) - train_size

train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]


In [270]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)


In [271]:
trainX.shape, trainY.shape, testX.shape, testY.shape

((34, 1), (34,), (14, 1), (14,))

In [272]:
#Reshape the dataset size

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0],1,testX.shape[1]))

LSTM MODEL
Model with 2 layers with 8 and 4 units as input and 1 layer of output.
Epochs=500
Batch Size = 4 unit

In [273]:
model = Sequential()
model.add(LSTM(8,return_sequences=True, input_shape=(1,look_back)))
model.add(LSTM(4,return_sequences=False, input_shape=(1,look_back)))

model.add(Dense(1))

#compile the model

model.compile(loss='mean_squared_error', optimizer='rmsprop')

#Train the model
model.fit(trainX, trainY, epochs=50, batch_size=4, verbose=2)


Epoch 1/50
9/9 - 5s - loss: 0.6073 - 5s/epoch - 529ms/step
Epoch 2/50
9/9 - 0s - loss: 0.5653 - 35ms/epoch - 4ms/step
Epoch 3/50
9/9 - 0s - loss: 0.5315 - 32ms/epoch - 4ms/step
Epoch 4/50
9/9 - 0s - loss: 0.4999 - 32ms/epoch - 4ms/step
Epoch 5/50
9/9 - 0s - loss: 0.4682 - 38ms/epoch - 4ms/step
Epoch 6/50
9/9 - 0s - loss: 0.4348 - 34ms/epoch - 4ms/step
Epoch 7/50
9/9 - 0s - loss: 0.4010 - 31ms/epoch - 3ms/step
Epoch 8/50
9/9 - 0s - loss: 0.3675 - 32ms/epoch - 4ms/step
Epoch 9/50
9/9 - 0s - loss: 0.3336 - 31ms/epoch - 3ms/step
Epoch 10/50
9/9 - 0s - loss: 0.2992 - 34ms/epoch - 4ms/step
Epoch 11/50
9/9 - 0s - loss: 0.2657 - 32ms/epoch - 4ms/step
Epoch 12/50
9/9 - 0s - loss: 0.2339 - 31ms/epoch - 3ms/step
Epoch 13/50
9/9 - 0s - loss: 0.2027 - 33ms/epoch - 4ms/step
Epoch 14/50
9/9 - 0s - loss: 0.1722 - 36ms/epoch - 4ms/step
Epoch 15/50
9/9 - 0s - loss: 0.1443 - 34ms/epoch - 4ms/step
Epoch 16/50
9/9 - 0s - loss: 0.1191 - 32ms/epoch - 4ms/step
Epoch 17/50
9/9 - 0s - loss: 0.0962 - 36ms/epoch 

# Run the predictions

In [274]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

1/1 [==============================] - 0s 20ms/step


In [275]:
trainPredict= scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [276]:
trainY[0], trainPredict[:,0]

(array([ 982., 1118., 1122.,  959.,  919.,  997.,  959., 1050.,  434.,
         884., 1046., 1213., 1128.,  877., 1039., 1162., 1168.,  906.,
        1166., 1138., 1106.,  949., 1070., 1017., 1159., 1065., 1152.,
        1097., 1219., 1073., 1186., 1106., 1165., 1019.]),
 array([1024.8848 , 1036.4701 , 1077.3057 , 1078.4888 , 1029.4652 ,
        1017.2319 , 1041.0255 , 1029.4652 , 1057.0265 ,  869.6147 ,
        1006.48395, 1055.8243 , 1105.0708 , 1080.2611 , 1004.3305 ,
        1053.7184 , 1090.2539 , 1092.008  , 1013.244  , 1091.4237 ,
        1083.2094 , 1073.7496 , 1026.4126 , 1063.0223 , 1047.0815 ,
        1089.3759 , 1061.5256 , 1087.3241 , 1071.076  , 1106.7997 ,
        1063.9196 , 1097.2532 , 1073.7496 , 1091.1313 ], dtype=float32))

In [277]:
testY[0], testPredict[:,0]

(array([ 988., 1192.,  996., 1070., 1159., 1162.,  975., 1056.,  990.,
        1088.,  959., 1094., 1080., 1073.]),
 array([1060.3273, 1038.2935, 1098.9957, 1040.7219, 1063.0223, 1089.3759,
        1090.2539, 1034.3408, 1058.8278, 1038.901 , 1068.3969, 1029.4652,
        1070.1836, 1066.0107], dtype=float32))

In [278]:
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' %(trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' %(testScore))


Train Score: 140.70 RMSE
Test Score: 81.06 RMSE


In [279]:
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back: len(trainPredict) + look_back, :] = trainPredict

In [280]:
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict) + look_back : len(dataset)-1, :] = testPredict

In [281]:
list_Graph = []
list_Graph.append(go.Scatter(x=list(range(0,len(dataset))),y=scaler.inverse_transform(dataset).flatten(),name="Original Dataset"))
list_Graph.append(go.Scatter(x=list(range(0,len(trainPredictPlot))),y=trainPredictPlot.flatten(),name="Predict with Train set"))
list_Graph.append(go.Scatter(x=list(range(0,len(testPredictPlot))),y=testPredictPlot.flatten(),name="Predict with Test set"))


fig = go.Figure(data=list_Graph)
fig.update_layout(width=1024,height=500,title='[LSTM] Prediction for \n' + dict_zone[AREA], xaxis=dict(title='Month'),yaxis=dict(title='Number of Crimes'))
fig.show()